In [43]:
import pyFAI
from pyFAI.geometry import Geometry
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
import matplotlib.pyplot as plt
from skbeam.io.fit2d_save import fit2d_save
import numpy as np
from tifffile.tifffile import imread
import os
import scipy.stats as sts

In [44]:
# Define some functions for masking
def margin_mask(img_shape, edge_size):
    """
    Mask the edge of an image

    Parameters
    -----------
    img_shape: tuple
        The shape of the image
    edge_size: int
        Number of pixels to mask from the edge

    Returns
    --------
    2darray:
        The mask array, bad pixels are 0
    """
    mask = np.zeros(img_shape, dtype=bool)
    mask[edge_size:-edge_size, edge_size:-edge_size] = True
    return mask


def ring_blur_mask(img, q, alpha, bins, mask=None):
    """
    Perform a annular mask, which checks the ring statistics and masks any
    pixels which have a value greater or less than alpha * std away from the
    mean

    Parameters
    ----------
    img: 2darray
        The  image
    q: 2darray
        The  array which maps pixels to Q space
    alpha: float or tuple or, 1darray
        Then number of acceptable standard deviations, if tuple then we use
        a linear distribution of alphas from alpha[0] to alpha[1], if array
        then we just use that as the distribution of alphas
    mask: 2darray
        A starting flattened mask

    Returns
    --------
    2darray:
        The mask
    """

    if mask is None:
        mask = np.ones(img.shape).astype(bool)
    if mask.shape != img.shape:
        mask = mask.reshape(img.shape)
    msk_img = img[mask]
    msk_q = q[mask]

    int_q = np.digitize(q, bins[:-1], True) - 1
    # integration
    mean = sts.binned_statistic(msk_q, msk_img, bins=bins,
                                statistic='mean')[0]
    std = sts.binned_statistic(msk_q, msk_img, bins=bins,
                               statistic=np.std)[0]
    if type(alpha) is tuple:
        alpha = np.linspace(alpha[0], alpha[1], len(std))
    threshold = alpha * std
    lower = mean - threshold
    upper = mean + threshold

    # single out the too low and too high pixels
    too_low = img < lower[int_q]
    too_hi = img > upper[int_q]

    mask *= ~too_low * ~too_hi
    return mask.astype(bool)

In [45]:
# load the pyFAI stuff
geo = Geometry()
geo.setFit2D(
    directDist=25,
    tilt=.5,
    tiltPlanRotation=.5,
    centerX=1024,
    centerY=1024,
    pixelX=200,
    pixelY=200,
)

Detector Detector	 Spline= None	 PixelSize= 2.000e-04, 2.000e-04 m
SampleDetDist= 2.499905e-02m	PONI= 2.047981e-01, 2.045818e-01m	rot1=-0.008726  rot2= 0.000076  rot3= -0.000000 rad
DirectBeamDist= 25.000mm	Center: x=1024.000, y=1024.000 pix	Tilt=0.500 deg  tiltPlanRotation= 0.500 deg

In [46]:
# make the radial array
r = geo.rArray((2048, 2048))
pixel_size = [getattr(geo, a) for a in ['pixel1', 'pixel2']]
rres = np.hypot(*pixel_size)
rbins = np.arange(np.min(r) - rres / 2., np.max(r) + rres / 2., rres)

In [47]:
# put your image path in here
path = os.path.expanduser('~/shared/PDFdata/July/X17A/1/FinalSum_1.tif')
img = tifffile.imread(path)
img = np.flipud(img)

In [48]:
margin = 30
lower_thresh = 0
upper_thresh = None
alpha = 2.5
tmsk = None

In [49]:
if tmsk is None:
    tmsk = np.ones(img.shape, dtype=int).astype(bool)
if margin:
    tmsk *= margin_mask(img.shape, margin)
if lower_thresh:
    tmsk *= (img > lower_thresh).astype(bool)
if upper_thresh:
    tmsk *= (img < upper_thresh).astype(bool)
if alpha:
    tmsk *= ring_blur_mask(img, r, alpha, rbins, mask=tmsk)

/home/christopher/mc/envs/dp/lib/python3.5/site-packages/ipykernel/__main__.py:67: RuntimeWarning: invalid value encountered in less
/home/christopher/mc/envs/dp/lib/python3.5/site-packages/ipykernel/__main__.py:68: RuntimeWarning: invalid value encountered in greater


In [50]:
masked_img = img.copy()
masked_img[np.where(~tmsk)] = np.nan

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=True, sharey=True)
for ax, data in zip(axs, [img, tmsk, masked_img]):
    ax.imshow(data, cmap='viridis')
plt.show()